# **Data pipeline**
<a target="_blank" href="https://colab.research.google.com/github/raphaelrubrice/scVAE_mva2025/blob/main/data_pipeline.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## **Colab setup**

In [1]:
!git clone https://github.com/raphaelrubrice/scVAE_mva2025.git
%cd scVAE_mva2025
!pip install -r requirements.txt

Cloning into 'scVAE_mva2025'...
remote: Enumerating objects: 221, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 221 (delta 39), reused 50 (delta 19), pack-reused 131 (from 1)
Receiving objects: 100% (221/221), 944.69 KiB | 14.53 MiB/s, done.
Resolving deltas: 100% (103/103), done.
/content/scVAE_mva2025
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.0/183.0 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.1/284.1 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 113.0 MB/s eta 0:00:00


### Test Downloads

In [7]:
from data_pipeline.src.downloader import run_downloads

run_downloads()


Saving extracted PBMC datasets to: /content/scVAE_mva2025/data_pipeline/data/pbmc_raw

✓ Already downloaded/extracted: /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD34
------------------------------------------------------------
✓ Already downloaded/extracted: /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD19_B
------------------------------------------------------------
✓ Already downloaded/extracted: /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD56_NK
------------------------------------------------------------
✓ Already downloaded/extracted: /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD4_helper
------------------------------------------------------------
✓ Already downloaded/extracted: /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD4_CD25
------------------------------------------------------------
✓ Already downloaded/extracted: /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD4_CD45RA_CD25neg
--------------------------------------------------

In [3]:
from data_pipeline.src.config import DATASETS
from data_pipeline.src.load_anndata import load_anndata
from pathlib import Path

folder = Path("data_pipeline/data/pbmc_raw/CD4_CD45RO")
meta = DATASETS["CD4_CD45RO"]

adata = load_anndata(folder, meta)
adata

Loading 10x data from: data_pipeline/data/pbmc_raw/CD4_CD45RO/filtered_matrices_mex/hg19
✓ Loaded 10224 cells × 32738 genes from CD4_CD45RO


AnnData object with n_obs × n_vars = 10224 × 32738
    obs: 'cell_type_lvl1', 'cell_type_lvl2', 'cell_type_lvl3', 'cell_type_lvl4', 'dataset', 'cell_type_lvl1__code', 'cell_type_lvl2__code', 'cell_type_lvl3__code', 'cell_type_lvl4__code'
    var: 'gene_ids'

In [4]:
from data_pipeline.src.combine import run_combine

combined, collection = run_combine(
    do_write_shards=True,
    write_combined=True,
    harmonize_var=False
)

Loading all PBMC AnnData objects...
📦 Loading dataset: CD34
Loading 10x data from: /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD34/filtered_matrices_mex/hg19
✓ Loaded 9232 cells × 32738 genes from CD34
📦 Loading dataset: CD19_B
Loading 10x data from: /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD19_B/filtered_matrices_mex/hg19
✓ Loaded 10085 cells × 32738 genes from CD19_B
📦 Loading dataset: CD56_NK
Loading 10x data from: /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD56_NK/filtered_matrices_mex/hg19
✓ Loaded 8385 cells × 32738 genes from CD56_NK
📦 Loading dataset: CD4_helper
Loading 10x data from: /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD4_helper/filtered_matrices_mex/hg19
✓ Loaded 11213 cells × 32738 genes from CD4_helper
📦 Loading dataset: CD4_CD25
Loading 10x data from: /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD4_CD25/filtered_matrices_mex/hg19
✓ Loaded 10263 cells × 32738 genes from CD4_CD25
📦 Loading dataset: CD4_CD45RA_CD25neg
Loading 

/usr/local/lib/python3.12/dist-packages/anndata/_core/anndata.py:1796: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


✓ Combined AnnData written to /content/scVAE_mva2025/data_pipeline/data/pbmc_processed/pbmc_combined.h5ad
✓ Label maps saved to /content/scVAE_mva2025/data_pipeline/data/pbmc_processed/label_maps.json
Building in‑memory AnnCollection...


/usr/local/lib/python3.12/dist-packages/legacy_api_wrap/__init__.py:88: UserWarning: Observation names are not unique.
  return fn(*args_all, **kw)


✓ AnnCollection constructed from 9 shards.

Combine stage complete!



In [5]:
import scanpy as sc
adata_combined = sc.read_h5ad("data_pipeline/data/pbmc_processed/pbmc_combined.h5ad")
adata_combined

/usr/local/lib/python3.12/dist-packages/anndata/_core/anndata.py:1796: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AnnData object with n_obs × n_vars = 92043 × 32738
    obs: 'cell_type_lvl1', 'cell_type_lvl2', 'cell_type_lvl3', 'cell_type_lvl4', 'dataset', 'cell_type_lvl1__code', 'cell_type_lvl2__code', 'cell_type_lvl3__code', 'cell_type_lvl4__code'
    var: 'gene_ids'
    uns: 'label_maps'

In [6]:
from data_pipeline.src.dataloader import build_dataloaders

train_loader, val_loader, test_loader = build_dataloaders(
    shard_dir="data_pipeline/data/pbmc_processed/shards",
    label_maps_path="data_pipeline/data/pbmc_processed/label_maps.json",
    batch_size=512,
    one_hot=True,
)

for i, batch in enumerate(train_loader):
        print("Example batch shapes:")
        print("X:", batch["X"].shape, "y1:", batch["y1"].shape, "y4:", batch["y4"].shape)
        break

/usr/local/lib/python3.12/dist-packages/legacy_api_wrap/__init__.py:88: UserWarning: Observation names are not unique.
  return fn(*args_all, **kw)


✓ Built AnnCollection with 92043 total cells.
Split: train=74554, val=8283, test=9206
✓ Train=74554, Val=8283, Test=9206
Example batch shapes:
X: torch.Size([512, 1, 32738]) y1: torch.Size([512, 1, 2]) y4: torch.Size([512, 1, 9])


In [44]:
from mixture_vae.mvae import ind_MoMVAE

from mixture_vae.distributions import (
    NormalDistribution,
    UniformDistribution,
    NegativeBinomial
)

from mixture_vae.training import (
    training_mvae
)

import torch

In [45]:
model = ind_MoMVAE(
    [
        {
          "input_dim": batch["X"].shape[-1],
          "hidden_dim": 20,
          "n_components": K,
          "n_layers": 3,
          "prior_latent": NormalDistribution(
              {
              "mu": torch.zeros((1, 20)),
              "std": torch.ones((1, 20))
              }
          ),
          "prior_input": NegativeBinomial(
              {
                "p":  0.5 * torch.ones((1, batch["X"].shape[-1])),
                "r": torch.mean(batch["X"], dim=0).reshape(1,-1)
              }
          ),
          "prior_categorical": UniformDistribution(
              {
                "a": torch.zeros((1, K)),
                "b": torch.ones((1, K))
              }
          ),
          "posterior_latent": NormalDistribution(
              {
              "mu": torch.zeros((1, 20)),
              "std": torch.ones((1, 20))
              }
          ),
        }
        for K in [2, 4, 8]
    ]
)

In [46]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

EPOCHS = 50
BETA_KL = 0.5
WARMUP = None
PATIENCE = 5
TOL = 5e-3

In [47]:
model, losses, all_parts, clusters, all_betas = training_mvae(
    dataloader=train_loader,
    val_dataloader=val_loader,
    model=model,
    optimizer=optimizer,
    epochs=EPOCHS,
    beta_kl=BETA_KL,
    patience=PATIENCE,
    tol=TOL,
    model_type=1
)

TRAINING:   0%|          | 0/50 [00:37<?, ?it/s]


KeyError: 0